In [79]:
import requests_cache

import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import date, timedelta
import os
import csv
from datetime import datetime
from dateutil import parser

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

In [80]:
requests_cache.install_cache('demo_cache',expire_after=None,allowable_methods=['GET'])
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been 

In [81]:
link = "https://www.worldometers.info/coronavirus/"

driver.get(link)

def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

In [82]:
header_by_day = ["#","Country","Date","TotalCases","NewCases","TotalDeaths","NewDeaths","TotalRecovered","NewRecovered",
"ActiveCases","Serious,Critical","Tot Cases/1M pop","Deaths/1M pop","TotalTests","Tests/1M pop","Population","Continent"]

header_by_week = ["#", "Country,Other", "Cases in the last 7 days", "Cases in the preceding 7 days",
"Weekly case /%/ change", "Cases in the last 7 days/1M pop", "Deaths in the last 7 days", 
"Deaths in the preceding 7 days", "Weekly Death /%/ change", "Deaths in the last 7 days/1M pop", "Population", "Continent"]

In [83]:
row_total = driver.find_elements(By.XPATH,"//tr[@class='even'] | //tr[@class='odd']")

rows_past = []
rows_past_modify = []
rows_yesterday = []
rows_two_days_ago = []
count = 0
row_count = 226

for n in row_total:
    n = n.get_attribute('textContent')
    if count == row_count:
        rows_past.append(n.split("\n"))
    else:
        count+=1

for n in rows_past:
    n.pop(0)
    n = n[: len(n) - 6]
    rows_past_modify.append(n)

rows_yesterday,rows_two_days_ago = split_list(rows_past_modify)

rows_yesterday = [ele for ele in rows_yesterday if ele != []]
rows_two_days_ago = [ele for ele in rows_two_days_ago if ele != []]

In [84]:
today = date.today()
yesterday = (today - timedelta(1)).strftime("%d-%m-%Y")
two_days_ago = (today - timedelta(2)).strftime("%d-%m-%Y")

for row in rows_yesterday:

    row.insert(2,yesterday)
for row in rows_two_days_ago:

    row.insert(2,two_days_ago)

total_covid = rows_two_days_ago + rows_yesterday 

Check xem có dòng nào bị dư giá trị rỗng không, nếu có cần chạy lại từ đầu

In [85]:
for i in total_covid:
    if len(i) > 17:
        if i[-1] == '' or i[-1] == ' ':
            i.pop(-1)
        else:
            print(i)

In [86]:
df_covid = pd.DataFrame(data=rows_yesterday,columns=header_by_day)
if not os.path.isfile('Corona_by_day.csv'):
    df_covid = pd.DataFrame(data=total_covid,columns=header_by_day)
    df_covid.to_csv("Corona_by_day.csv", sep = ",", mode = "a", index = False)
else:
    df_covid.to_csv("Corona_by_day.csv", sep = ",", mode = "a", index = False,header=False)

In [87]:
total_covid_df = pd.read_csv('Corona_by_day.csv')

In [88]:
def convert_numeric(num):
    condition = {",":"","+":""}
    if num == "nan" or num == " " or num == "  " or num == " N/A":
        return 0

    for i, j in condition.items():
        num = num.replace(i, j)
    return int(num)
for i in range(3,16):
    total_covid_df[header_by_day[i]] = total_covid_df[header_by_day[i]].apply(lambda x:convert_numeric(str(x)))
    
total_covid_df[header_by_day[16]] = total_covid_df[header_by_day[16]].apply(lambda x:str(x).replace(" ",""))
total_covid_df['Date'] = pd.to_datetime(total_covid_df['Date'],format="%d-%m-%Y")

In [89]:
total_covid_df

#                                   170
Country                    Burkina Faso
Date                2022-02-27 00:00:00
TotalCases                        20751
NewCases                              0
TotalDeaths                         375
NewDeaths                             0
TotalRecovered                    20309
NewRecovered                          0
ActiveCases                          67
Serious,Critical                      0
Tot Cases/1M pop                    950
Deaths/1M pop                        17
TotalTests                       248995
Tests/1M pop                      11394
Population                     21852404
Continent                        Africa
Name: 169, dtype: object

In [90]:
total_covid_df.dtypes

#                            int64
Country                     object
Date                datetime64[ns]
TotalCases                   int64
NewCases                     int64
TotalDeaths                  int64
NewDeaths                    int64
TotalRecovered               int64
NewRecovered                 int64
ActiveCases                  int64
Serious,Critical             int64
Tot Cases/1M pop             int64
Deaths/1M pop                int64
TotalTests                   int64
Tests/1M pop                 int64
Population                   int64
Continent                   object
dtype: object